In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
from functools import reduce

url = 'https://books.toscrape.com/'
print(url)

https://books.toscrape.com/


In [2]:
response = requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')

In [3]:
lst = (soup.find('ul', attrs = {'class':''}).find_all('li'))
links = []
for n in range(len(lst)):
    link = soup.find('ul', attrs = {'class':''}).find_all('li')[n].find('a').get('href')
    domain = url
    full_link = domain+link
    links.append(full_link)
links

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'https://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'https://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'https://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'https://books.toscrape.com/catalogue/category/books/religion_12/index.html',
 'https://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html',
 'https://books.toscrape.com/catalogue

In [11]:
rate_dict = {'One':1, 'Two' : 2 , 'Three' : 3 , 'Four' : 4 , 'Five' : 5}
for url in links:
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    for book in soup.find_all('article',attrs={'class':'product_pod'}):
        pricex=float(book.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
        titlex=book.find('h3').find('a').get('title').strip()
        ratex=rate_dict[book.find('p').get('class')[1]]
        categoryx=soup.find('div',attrs={'class':'page-header action'}).find('h1').get_text()
        connection = sqlite3.connect('book_list.db')
        cursor = connection.cursor()
        cursor.execute('insert into books2 (book,category,price,rate) values (? ,? ,? ,? )',[titlex,categoryx,pricex,ratex])
        connection.commit()
        connection.close()
    while soup.find('li',attrs={'class':'next'}):
        new_page_url=reduce(lambda a,b:a+'/'+b,url.split('/')[:-1])+'/'+soup.find('li',attrs={'class':'next'}).find('a').get('href')
        soup=BeautifulSoup(requests.get(new_page_url).text,'html.parser')
        for book in soup.find_all('article',attrs={'class':'product_pod'}):
            pricex=float(book.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
            titlex=book.find('h3').find('a').get('title').strip()
            ratex=rate_dict[book.find('p').get('class')[1]]
            categoryx=soup.find('div',attrs={'class':'page-header action'}).find('h1').get_text()
            connection = sqlite3.connect('book_list.db')
            cursor = connection.cursor()
            cursor.execute('insert into books2 (book,category,price,rate) values (? ,? ,? ,? )',[titlex,categoryx,pricex,ratex])
            connection.commit()
            connection.close()
        

In [15]:
import pandas as pd
connection = sqlite3.connect('book_list.db')
df = pd.read_sql('select * from books2', connection)
df

,ID,book,category,price,rate
0,1,It's Only the Himalayas,Travel,45.17,2
1,2,Full Moon over Noahâs Ark: An Odyssey to Mou...,Travel,49.43,4
2,3,See America: A Celebration of Our National Par...,Travel,48.87,3
3,4,Vagabonding: An Uncommon Guide to the Art of L...,Travel,36.94,2
4,5,Under the Tuscan Sun,Travel,37.33,3
...,...,...,...,...,...
995,996,Why the Right Went Wrong: Conservatism--From G...,Politics,52.65,4
996,997,Equal Is Unfair: America's Misguided Fight Aga...,Politics,56.86,1
997,998,Amid the Chaos,Cultural,36.58,1
998,999,Dark Notes,Erotica,19.19,5
